In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))
os.chdir("../")

In [2]:
import os
import json
import logging

import numpy as np
import torch
from sklearn.neighbors import NearestNeighbors
from torch import Tensor

from graphcast_utils import (
    add_edge_features,
    add_node_features,
    cell_to_adj,
    create_graph,
    create_heterograph,
    get_edge_len,
    latlon2xyz,
    xyz2latlon,
    find_subset_indices,
)

/vol/bitbucket/bet20/nlam/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Multiscale Graph

In [3]:
big_dataset = "era5_uk_big"
small_dataset = "era5_uk_small"

big_grid_xy = np.load(f"data/{big_dataset}/static/nwp_xy.npy")
small_grid_xy = np.load(f"data/{small_dataset}/static/nwp_xy.npy")